One of my colleagues, Sophie Searcy, recently wrote an [blog post](https://soph.info/2019/05/07/imbalance/) that dealt with imbalanced classes. She looked at some of the ways that you can address an imbalanced learning problem, and some of the pros and cons of each. One of the big takeaways of that article (which you should read!) was to carefully consider whether or not you should address the problem of imbalance by oversampling, or if you should look at some of the alternatives: adjusting the weights of the classes or even checking to see if your model deals with imbalanced data naturally.

This article is about how to do cross-validation once you have decided that oversampling is the right approach for your problem. If you prefer to We will be using a processed thyroid dataset

In [1]:
from imblearn.pipeline import make_pipeline
from imblearn import datasets
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, KFold
from sklearn.metrics import recall_score, roc_auc_score

import pandas as pd
import numpy as np

## Load some data

We are going to load the `thyroid` dataset from imbalanced-learn's collection. This is a classification task where we are trying to determine whether someone has a bad thyroid or not.

Imbalanced-learn's datasets are typically labelled `-1` (the majority class) and `1` (the rare class we are trying to find). Let's change that to `0` and `1`, which we are more used to for binary classification

In [2]:
thyroid_collection = datasets.fetch_datasets()['thyroid_sick']
X = pd.DataFrame(thyroid_collection['data'])
y = thyroid_collection['target']
y[y==-1] = 0

What is the imbalance? We can ask what fraction of the `y` values are `1` (which is just the mean of values in `y`)

In [3]:
# What is the imbalance?
y.mean()

0.0612407211028632

Okay, about 6% of our patients have thyroid issues. Another way of thinking about this is that we have approximately 15 people with healthy thyroids for every person with a bad thyroid.

Our goal will be to find a good recall (i.e. we want our classifier to find as many positive cases as it can). We have to be aware there is a danger in using this metric, as simply predicting _everyone_ has a bad thyroid will make the recall 100%. 

In practice, you might want to optimize some other metric like AUC and then choose a threshold to optimize recall once you have selected a model. Because AUC can be confusing, I am using the slightly more dangerous recall as my metric in this problem.

## Standardizing our splits

Let's make sure that our results are consistent as we try different methods. It is a little simpler to have `cv=5` in all of our grid searches and cross-validations, but we will get different splits each time.

If we use `cv=kf`, where `kf` is a `KFold` object we can ensure that we get the same splits each time.

In [4]:
kf = KFold(n_splits=5, random_state=42, shuffle=False)

## Standard approach, no oversampling

Let's get a baseline result by picking a random forest. 

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=45)

rf = RandomForestClassifier(n_estimators=100, random_state=13)

cross_val_score(rf, X_train, y_train, cv=kf, scoring='roc_auc')

array([0.99006284, 0.9943431 , 0.99762992, 0.99316292, 0.99746323])

These are pretty decent result, and we haven't even optimized the model. 

Let's try to do some hyperparameter tuning:

In [53]:
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 6, 10, 12],
    'random_state': [13]
}

grid_no_up = GridSearchCV(rf, param_grid=params, cv=kf, 
                          scoring='recall').fit(X_train, y_train)

In [54]:
grid_no_up.best_score_

0.7803820054409211

In [55]:
grid_no_up.cv_results_['mean_test_score']

array([0.37592633, 0.16071796, 0.08487883, 0.64779239, 0.65553493,
       0.62892524, 0.77319976, 0.76886478, 0.78038201, 0.77010354,
       0.77497469, 0.77784313])

In [56]:
grid_no_up.best_params_

{'max_depth': 10, 'n_estimators': 200, 'random_state': 13}

Ok, we see that we have about 78% recall on one of our models before we have tried oversampling. This is the number to beat.

Normally we would wait until we had finished our modeling to look at the test set, but an important part of this is to see how oversampling, done incorrectly, can make us too confident in our ability to generalize based off cross-validation. We haven't oversampled yet, so let's just check that the test scores are in line with what we expect from the CV scores about (i.e. about 78%)

In [57]:
recall_score(y_test, grid_no_up.predict(X_test))

0.8035714285714286

Ok, this looks like it is (roughly) consistent with the CV results.

## Oversampling (the wrong way)

Let's just oversample the training data (we are smart enough not to oversample the test data)

In [11]:
X_train_upsample, y_train_upsample = SMOTE(random_state=42).fit_sample(X_train, y_train)

Let's check that upsampling gave us an even split of the two classes:

In [12]:
y_train_upsample.mean()

0.5

Ok, let's cross-validate using grid search. Remember the training set has been upsampled, that is not being done as part of the GridSearch

In [13]:
params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [4, 6, 10, 12],
    'random_state': [13]
}

grid_naive_up = GridSearchCV(rf, param_grid=params, cv=kf, 
                             scoring='recall').fit(X_train_upsample, 
                                                   y_train_upsample)
grid_naive_up.best_score_

0.9843160927198451

This is an amazing recall! If we look at the validation scores, they _all_ look pretty good:

In [14]:
grid_naive_up.cv_results_['mean_test_score']

array([0.93360792, 0.9345499 , 0.93337591, 0.94714925, 0.94736138,
       0.94273667, 0.97585677, 0.98218414, 0.97864618, 0.98237253,
       0.98187974, 0.98431609])

Here is the model that made these results:

In [15]:
grid_naive_up.best_params_

{'max_depth': 12, 'n_estimators': 200, 'random_state': 13}

Ok, let's look at how it does on the training set as a whole (once we eliminate the upsampling)

In [16]:
recall_score(y_train, grid_naive_up.predict(X_train))

1.0

Ok, what about the test set?

In [17]:
# But wait ... uh-oh, spagetti-os!
recall_score(y_test, grid_naive_up.predict(X_test))

0.9107142857142857

Ok, time for some good news/bad news:
- good: the recall on the test set is 91%, better than the 80% we got without upsampling
- bad: our confidence in the cross-valdation results went down. With no upsampling, the validation recall was 78%, which was a good estimate of the test validation of 80%. With upsampling, the validation recall was 100% which isn't a good measure of the test recall (91%)

## Let's make SMOTE-ing part of our cross validation!

The issue is that we 
- oversample
- the split into cross-validation folds

You can imagine the simplest method of over-sampling (namely, copying the data point). Let's say every data point is copied 6 times. It isn't hard to imagine that in 3-fold validation, a typical folding has (on average) 2 copies of each point in each fold. If the classifier memories the test set, the validation set will get a perfect score because the validation set has no new data points!

Instead, we should split into training and validation folds. Then, on each fold, we should
1. Oversample the minority class
2. Train the classifier on the training folds
3. Validate the classifier on the remaining fold

Let's see this in detail

In [18]:
example_params = {
        'n_estimators': 100,
        'max_depth': 5,
        'random_state': 13
    }

def score_model(model, params, cv=None):
    if cv is None:
        cv = KFold(n_splits=5, random_state=42)

    smoter = SMOTE(random_state=42)
    
    scores = []

    for train_fold_index, val_fold_index in cv.split(X_train, y_train):
        X_train_fold, y_train_fold = X_train.iloc[train_fold_index], y_train[train_fold_index]
        X_val_fold, y_val_fold = X_train.iloc[val_fold_index], y_train[val_fold_index]

        X_train_fold_upsample, y_train_fold_upsample = smoter.fit_resample(X_train_fold,
                                                                           y_train_fold)
        model_obj = model(**params).fit(X_train_fold_upsample, y_train_fold_upsample)
        score = recall_score(y_val_fold, model_obj.predict(X_val_fold))
        scores.append(score)
    return np.array(scores)

In [19]:
score_model(RandomForestClassifier, example_params, cv=kf)

array([0.78378378, 0.76315789, 0.96875   , 0.81481481, 0.90243902])

Janky grid search:

In [20]:
params

{'n_estimators': [50, 100, 200],
 'max_depth': [4, 6, 10, 12],
 'random_state': [13]}

In [21]:
score_tracker = []
for n_estimators in params['n_estimators']:
    for max_depth in params['max_depth']:
        example_params = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'random_state': 13
        }
        example_params['recall'] = score_model(RandomForestClassifier, 
                                               example_params, cv=kf).mean()
        score_tracker.append(example_params)

In [22]:
sorted(score_tracker, key=lambda x: x['recall'], reverse=True)

[{'n_estimators': 50,
  'max_depth': 4,
  'random_state': 13,
  'recall': 0.8486884268736002},
 {'n_estimators': 100,
  'max_depth': 6,
  'random_state': 13,
  'recall': 0.8291411035554168},
 {'n_estimators': 50,
  'max_depth': 12,
  'random_state': 13,
  'recall': 0.8283643801053943},
 {'n_estimators': 200,
  'max_depth': 12,
  'random_state': 13,
  'recall': 0.8254883333269085},
 {'n_estimators': 50,
  'max_depth': 10,
  'random_state': 13,
  'recall': 0.825245471723328},
 {'n_estimators': 200,
  'max_depth': 6,
  'random_state': 13,
  'recall': 0.82387794566068},
 {'n_estimators': 100,
  'max_depth': 12,
  'random_state': 13,
  'recall': 0.8224316180750713},
 {'n_estimators': 50,
  'max_depth': 6,
  'random_state': 13,
  'recall': 0.8196299476626819},
 {'n_estimators': 100,
  'max_depth': 4,
  'random_state': 13,
  'recall': 0.8172428365464309},
 {'n_estimators': 200,
  'max_depth': 4,
  'random_state': 13,
  'recall': 0.810992836546431},
 {'n_estimators': 100,
  'max_depth': 10,
  

The best estimator appears to have the parameters
```
{'n_estimators': 50,
  'max_depth': 4,
  'random_state': 13,
 }
```
and a recall score of 85% for the validation score. Let's see how this compares with the test score:

In [23]:
rf = RandomForestClassifier(n_estimators=50, max_depth=4, random_state=13)
rf.fit(X_train_upsample, y_train_upsample)
recall_score(y_test, rf.predict(X_test))

0.8392857142857143

Note that is is roughly consistent (84% vs 85%)

## Let's use the imbalanced class pipeline

The imbalanced-learn dataset extends the sklearn's built-in pipeline methods. Specifically, you can import 
```python
from sklearn.pipeline import Pipeline, make_pipeline
```
which will allow you to do multiple steps at once. It is also nice that if you _fit_ the model, all the steps (such as scaling, and the model) are fit at once. If you _predict_ with the model, scaling steps are only _trensformed_, so you can pass multiple steps into a pipeline. 

There is a restriction. The restriction comes partially from the naming of functions (e.g. `transform` vs `resample`) but one way of thing of it is that sklearn's pipeline only allows for one row in to be transformed to another row (perhaps with different or added features). To upsample, we need to _increase_ the number of rows. Imbalanced-learn generalizes the pipeline, but tries to keep the syntax and function names the same:
```python
from imblearn.pipeline import Pipeline, make_pipeline
```

Let's see it in action:

In [24]:
imba_pipeline = make_pipeline(SMOTE(random_state=42), 
                              RandomForestClassifier(n_estimators=100, random_state=13))

imba_pipeline

Pipeline(memory=None,
     steps=[('smote', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterio...ators=100, n_jobs=None,
            oob_score=False, random_state=13, verbose=0, warm_start=False))])

In [25]:
cross_val_score(imba_pipeline, X_train, y_train, scoring='recall', cv=kf)

array([0.75675676, 0.78947368, 0.90625   , 0.77777778, 0.7804878 ])

In [26]:
new_params = {'randomforestclassifier__' + key: params[key] for key in params}
grid_imba = GridSearchCV(imba_pipeline, param_grid=new_params, cv=kf, scoring='recall',
                        return_train_score=True)

In [27]:
grid_imba.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=False),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('smote', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterio...ators=100, n_jobs=None,
            oob_score=False, random_state=13, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'randomforestclassifier__n_estimators': [50, 100, 200], 'randomforestclassifier__max_depth': [4, 6, 10, 12], 'randomforestclassifier__random_state': [13]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall', verbose=0)

In [28]:
grid_imba.cv_results_['mean_test_score'], grid_imba.cv_results_['mean_train_score']

(array([0.84867805, 0.81722134, 0.81096913, 0.81961792, 0.82913244,
        0.82386742, 0.82525267, 0.80970919, 0.80956689, 0.82837268,
        0.8224292 , 0.82550424]),
 array([0.90125477, 0.8855561 , 0.87690479, 0.93433132, 0.92447245,
        0.91290858, 0.99573952, 0.99584802, 0.99584802, 0.99855072,
        0.99855072, 0.99855072]))

In [29]:
grid_imba.best_score_

0.8486780485230826

In [30]:
grid_imba.best_params_

{'randomforestclassifier__max_depth': 4,
 'randomforestclassifier__n_estimators': 50,
 'randomforestclassifier__random_state': 13}

In [31]:
y_test_predict = grid_imba.best_estimator_.predict(X_test)

In [32]:
recall_score(y_test, y_test_predict)

0.8392857142857143

Note that we are getting the same result we would get if we bypassed the SMOTE step altogether. When making a prediction, the SMOTE step does nothing:

In [33]:
y_test_predict = grid_imba.best_estimator_.named_steps['randomforestclassifier'].predict(X_test)

In [34]:
recall_score(y_test, y_test_predict)

0.8392857142857143

## What if I just trained a model with the "best" params?

In [35]:
snip_len = len('randomforestclassifier__')
best_params = {key[snip_len:] : grid_imba.best_params_[key] for key in grid_imba.best_params_}
best_params

{'max_depth': 4, 'n_estimators': 50, 'random_state': 13}

In [36]:
rf = RandomForestClassifier(**best_params).fit(X_train_upsample, y_train_upsample)
recall_score(y_test, rf.predict(X_test))

0.8392857142857143

In [38]:
rf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=13, verbose=0, warm_start=False)